<a href="https://colab.research.google.com/github/skyli42/CSC413S2021-Team-22/blob/master/CSC413_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install download
!python -c "from download import download; download('https://raw.githubusercontent.com/NeuroTechX/moabb/master/requirements.txt', 'requirements.txt', replace=True)"
!pip install -r requirements.txt
!rm requirements.txt
!pip install -U https://github.com/NeuroTechX/moabb/archive/master.zip

from moabb.datasets import BNCI2014001
from moabb.paradigms import (LeftRightImagery, MotorImagery,
                             FilterBankMotorImagery)

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/download/download.py", line 208, in _fetch_file
    u = urllib.request.urlopen(req, timeout=timeout)
  File "/usr/lib/python3.7/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/usr/lib/python3.7/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/usr/lib/python3.7/urllib/request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "/usr/lib/python3.7/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/usr/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.7/urllib/request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 404: Not Found

During handling of the above exception, another exception occurred:

Traceback (most recent call last

In [ ]:
!pip install braindecode

from braindecode.datasets.moabb import MOABBDataset
import mne
from braindecode.datautil.windowers import create_windows_from_events
from braindecode.datautil.preprocess import exponential_moving_standardize
from braindecode.datautil.preprocess import MNEPreproc, NumpyPreproc, preprocess
from sklearn.model_selection import train_test_split
from braindecode.datasets import moabb

     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 133kB 32.9MB/s 
  Created wheel for braindecode: filename=Braindecode-0.5-cp37-none-any.whl size=53741 sha256=fbc21f4ed531b9676d23e4949cab9202a82087e76aafa80620919960e8383dfb
  Stored in directory: /root/.cache/pip/wheels/2d/dc/b4/8db33baacda83683788e6f94a8a9dd5791bf3f65b17fd60c66
Successfully built braindecode


In [ ]:
import os

#!pip install numpy --upgrade
import numpy as np


from tqdm import tqdm

from scipy.signal import welch, spectrogram
from scipy.ndimage.interpolation import shift
from scipy.stats import kurtosis, skew
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader


# Feature Extraction

### Window Funciton

In [ ]:
def next_pos(data, window_size, overlap_size, seq_axis):
  sample_length = data.shape[seq_axis]
  pos = 0
  while (data.shape[seq_axis] > pos+window_size):
    yield pos 
    pos += window_size - overlap_size

def overlap_window(data, window_size, overlap_size, seq_axis):
  '''
  Takes data and creates overlapping windows. 

  Args:
      data:         Data in format: trial x channel x sequence
      window_size:  The size of the window in samples
      overlap_size: number of samples of overlap of current window with previous window
      seq_axis:     The axis the windowing is calculated on.

  Returns:
      The windowed data in format: trial x channel x windows x window_size
  '''

  sidx = []
  for i in next_pos(data, window_size, overlap_size, seq_axis):
    sidx.append(i)
  windows = []
  for i in sidx:#[1:]:
    windows.append(data[...,i:i+window_size, np.newaxis])
  
  windows= np.concatenate(windows, axis=-1)

  return np.swapaxes(windows, -1, -2) # trial x channel x window x sample

## Agument

### Add noise

In [ ]:
def add_noise(x, y, duplication_factor, noise_level):
  if duplication_factor == 0:
    return x, y
  z = np.repeat(x,duplication_factor, axis=0)
  w = np.repeat(y,duplication_factor, axis=0)
  for i in range(x.shape[0]):
    for j in range(x.shape[1]):
      mu = np.mean(x[i,j,:])
      std = np.std(x[i,j,:])*noise_level
      for k in range(1,duplication_factor):
        z[i*duplication_factor+k,j,:] = z[i,j,:] + np.random.normal(0, std, (x.shape[2]))
  return z, w

## Features

In [ ]:
def normalize_axis(data, ax) :
    mean = np.mean(data, axis=ax)
    float_data = data.astype(np.float64)
    mean_centered = float_data - mean[...,np.newaxis]
    std = np.std(mean_centered, axis=ax)
    normalized = mean_centered/std[...,np.newaxis]
    return normalized

### Power Spectral Density

Gamma (γ)	&gt;35 Hz	Concentration

Beta (β)	12–35 Hz	Anxiety dominant, active, external attention, relaxed

Alpha (α)	8–12 Hz	Very relaxed, passive attention

Theta (θ)	4–8 Hz	Deeply relaxed, inward focused

Delta (δ)	0.5–4 Hz	Sleep

In [ ]:
def psd_feature(windowed_data, normalize=False, sample_freq=250, cutoff_freq=60): #use 250Hz, 

  f, psd = welch(windowed_data, sample_freq, nperseg=windowed_data.shape[3], axis=-1) #use the full window size for nperseg


  #idx = np.argwhere(f<cutoff_freq) 
  #psd = psd[..., idx]               #select freq_bins less then cut off

  print("Calculating Freq bands (Alpha, Beta, ...ect)")
  Delta_idx = np.argwhere(f<4)
  Theta_idx = np.argwhere((4<=f) & (f<8))
  Alpha_idx = np.argwhere((8<=f) & (f<12))
  Beta_idx  = np.argwhere((12<=f) & (f<35))
  Gamma_idx = np.argwhere((35<=f) & (f<cutoff_freq))
  print("Calculating Delta power")
  Delta = np.sum(psd[...,Delta_idx], axis=3)
  print("Calculating Theta power")
  Theta = np.sum(psd[...,Theta_idx], axis=3)
  print("Calculating Alpha power")
  Alpha = np.sum(psd[...,Alpha_idx], axis=3)
  print("Calculating Beta power")
  Beta  = np.sum(psd[...,Beta_idx ], axis=3)
  print("Calculating Gama power")
  Gamma = np.sum(psd[...,Gamma_idx], axis=3)

  ppsd = np.concatenate((Delta, Theta, Alpha, Beta, Gamma), axis=3)

  ppsd = np.swapaxes(np.squeeze(ppsd), -1, -2) #put window as last dim

  if normalize:
    ppsd = normalize(ppsd, 3)
  

  return  ppsd# returns trial x channel x freq_bin x window

In [ ]:
#huh = psd_feature(Z)
#print(huh.shape)
#print(huh[0,0,0,:])

### Zero Crossings


In [ ]:
def zero_crossings(windowed_data, normalize=False):
  shifted = np.roll(windowed_data, 1, axis=3)
  signs = windowed_data[...,1:] * shifted[...,1:] <=0
  crossings = np.sum(signs, axis=3)
  crossings = np.squeeze(crossings)
  
  if normalize:
    crossings = normalize_axis(crossings, 2)
  
  return crossings

In [ ]:
#crossings = zero_crossings(Z, True)
#print(crossings.shape)
#print(crossings[0,1,:])

### Kurtosis

In [ ]:
def window_kurtosis(data, normalize = False):
  k = kurtosis(data, axis=3)

  if normalize:
    k = normalize_axis(k,2)
  
  return k

In [ ]:
#k = window_kurtosis(Z, True)
#print(k.shape)
#print(k[0,0,:])

### Abs under curve

In [ ]:
def abs_under_curve(windowed_data):
  abs_data = np.abs(windowed_data)
  return np.sum(abs_data, axis=3)

In [ ]:
#absdata = abs_under_curve(Z)
#print(absdata.shape)
#Eprint(absdata[0,0,:])

### Skewedness

In [ ]:
#skewedness = skew(Z, axis=3)
#print(skewedness.shape)
#print(skewedness[0,0,:])

### Peak-Peak

In [ ]:
def pkpk(windowed_data):
  pk = np.max(windowed_data, axis=3) - np.min(windowed_data, axis=3)
  return pk

In [ ]:
#p = pkpk(Z)
#print(p.shape)
#print(p[0,0,:])

## Extract Freatures

#Model

In [ ]:
from braindecode.models.functions import squeeze_final_output,square,safe_log
from braindecode.models.modules import Expression
from torch.nn import init
def init_weights(m):
    if type(m) == nn.Conv2d:
      init.xavier_uniform_(m.weight, gain=1)
      if (m.bias is not None):
        init.constant_(m.bias,0 )
    elif type(m)==nn.BatchNorm2d:
      init.constant_(m.weight,1)
      init.constant_(m.weight,1)
    elif type(m) == nn.Linear:
      init.xavier_uniform_(m.weight, gain=1)
      init.constant_(m.bias,0 )
      

class CNN(nn.Module):   
    def __init__(self,num_channels,n_classes):
        super(CNN, self).__init__()

        n_filters_time=40
        filter_time_length=25
        n_filters_spat=40
        pool_time_length=75
        pool_time_stride=15
        final_conv_length=69
        self.num_channels = num_channels
        self.cnn_layers = nn.Sequential(
            # Defining a 1D convolution layer IN (B,1,22,T)
            nn.Conv2d(1, n_filters_time, kernel_size=(filter_time_length,1)), #(B,50,22,T-9)
            nn.Conv2d(n_filters_time, n_filters_spat, kernel_size=(1,num_channels),bias=False), # (B,25,1,T)
            nn.BatchNorm2d(n_filters_spat, affine=True),
            Expression(square),
            nn.AvgPool2d(kernel_size=(pool_time_length,1),stride=(pool_time_stride,1)),
            Expression(safe_log),
            nn.Dropout(0.5),
            # nn.Flatten(),
            # nn.Linear(2760,n_classes),
            # nn.Linear(48240,n_channels),
            nn.Conv2d(n_filters_spat,n_classes,kernel_size=(final_conv_length, 1),bias=True,),
            nn.LogSoftmax(dim=1),
            Expression(squeeze_final_output)
        ) # output is (B,50,750/9)

        # self.linear_layers = nn.Sequential(
        #     nn.Linear(6050, 4)
        #     # nn.ReLU(),
        #     # nn.Linear(100,4)
        # )
        # self.softmax = nn.Softmax(dim=1)
        self.cnn_layers.apply(init_weights)

    # Defining the forward pass    
    def forward(self, x):
        x= torch.unsqueeze(x,-1)
        x = x.permute(0,3,2,1)
        x = self.cnn_layers(x)
        # print(x.shape)
        # x = x.view(x.size(0), -1)
        # x = self.linear_layers(x)
        # print(x.shape)
        # x= Expression(squeeze_final_output)(x)
        # x= x[:,:,0]
        # print(x.shape)
        # x = self.softmax(x)
        return x

In [ ]:
def extract_features(windowed_data):
  '''
  Takes windowed time series data and computes freqency and statistical features for each window. 

  Features computed are (25 total): power spectral density for 18 different frequecies, kurtosis, 
  abosulute area under curve, zero crossings, mean, varience, skewedness, and peak to peak
  
  Args:
      windowed_data:    Data in format: trials x channels x windows x window_size
  
  Returns:
      The windowed data in format: trials x channels x features x windows
  '''
  print("Calculating psd")
  psd = psd_feature(windowed_data)
  print("PSD shape: ", psd.shape)
  print("Calculating kertosis")
  k = window_kurtosis(windowed_data)
  print("Calculating Abs AUC")
  abs_under = abs_under_curve(windowed_data)
  print("Calculating zeros")
  zeros = zero_crossings(windowed_data)
  print("Calcualteing mean")
  mean = np.mean(windowed_data, axis=3)
  print("Calculating var")
  var = np.var(windowed_data, axis=3)
  print("Calculating skew")
  skewedness = skew(windowed_data, axis=3)
  print("Calculating pkpk")
  pk = pkpk(windowed_data)

  #Since these are only 1 feature the features axis needs to be created
  k = k[:,:,np.newaxis,:]                     
  abs_under = abs_under[:,:,np.newaxis,:]
  zeros = zeros[:,:,np.newaxis,:]
  mean = mean[:,:,np.newaxis,:]
  var = var[:,:,np.newaxis,:]
  skewedness = skewedness[:,:,np.newaxis,:]
  pk = pk[:,:,np.newaxis,:]

  print("cat")
  features = np.concatenate((psd,k), axis=2)
  features = np.concatenate((features, abs_under), axis=2)
  features = np.concatenate((features, zeros), axis=2)
  features = np.concatenate((features, mean), axis=2)
  features = np.concatenate((features, var), axis=2)
  features = np.concatenate((features, skewedness), axis=2)
  features = np.concatenate((features, pk), axis=2)

  return features

# Preprocess

In [ ]:
def our_preprocess(dataset):
  low_cut_hz = 4.  # low cut frequency for filtering
  high_cut_hz = 38.  # high cut frequency for filtering
  # Parameters for exponential moving standardization
  factor_new = 1e-3
  init_block_size = 1000

  preprocessors = [
      # keep only EEG sensors
      MNEPreproc(fn='pick_types', eeg=True, meg=False, stim=False),
      # convert from volt to microvolt, directly modifying the numpy array
      NumpyPreproc(fn=lambda x: x * 1e6),
      # bandpass filter
      MNEPreproc(fn='filter', l_freq=low_cut_hz, h_freq=high_cut_hz),
      # exponential moving standardization
      NumpyPreproc(fn=exponential_moving_standardize, factor_new=factor_new,
          init_block_size=init_block_size)
  ]

  # Transform the data
  preprocess(dataset, preprocessors)

In [ ]:
def feature_extract(subjects, windowsize, aug_factor, aug_noise_factor):
  
  dataset = MOABBDataset(dataset_name="BNCI2014001", subject_ids=subjects)

  our_preprocess(dataset)

  trial_start_offset_seconds = -0.5
  # Extract sampling frequency, check that they are same in all datasets
  sfreq = dataset.datasets[0].raw.info['sfreq']
  assert all([ds.raw.info['sfreq'] == sfreq for ds in dataset.datasets])
  # Calculate the trial start offset in samples.
  trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

  # Create windows using braindecode function for this. It needs parameters to define how
  # trials should be used.
  windows_dataset = create_windows_from_events(
      dataset,
      trial_start_offset_samples=trial_start_offset_samples,
      trial_stop_offset_samples=0,
      preload=True
  )

  splitted = windows_dataset.split('session')
  train_set = splitted['session_T']
  valid_set = splitted['session_E']

  return train_set,valid_set
  #Get the actual Data
  # X_train,y_train,X_val,y_val = [],[],[],[]
  # data,labels = [],[]
  # # labels = []
  # for d, l, g in windows_dataset:
  #   data.append(d)
  #   labels.append(l)
  # data = np.array(data)
  # labels = np.array(labels)
  # X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.3, random_state=1)

  # X_train, labels = add_noise(X_train, y_train, aug_factor, aug_noise_factor)
  # X_train = overlap_window(X_train, windowsize, windowsize//2, 2) # trial x channel x window x sample
  # # X_train = extract_features(X_train) #final shape trials x Channels x Feature x Window

  # X_val = overlap_window(X_val, windowsize, windowsize//2, 2) # trial x channel x window x sample
  # # X_val = extract_features(X_val) #final shape trials x Channels x Feature x Window

  # X_test = overlap_window(X_test, windowsize, windowsize//2, 2) # trial x channel x window x sample
  # # X_test = extract_features(X_test) #final shape trials x Channels x Feature x Window


  # return X_train, y_train, X_val, y_val

# Main

#Training

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch
X_val = torch.tensor(X_val)
y_val = torch.tensor(y_val)

valid_set = TensorDataset(X_val,y_val) # create your datset
val_dataloader = DataLoader(val_dataset) # create your dataloader

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [ ]:
import braindecode
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
import torch
from braindecode import EEGClassifier
# These values we found good for shallow network:
lr = 0.0625 * 0.01
weight_decay = 0

# For deep4 they should be:
# lr = 1 * 0.01
# weight_decay = 0.5 * 0.001
# for subject in range(1,10):
train_set, valid_set= feature_extract([1,2,3,4,5,6,7,8,9], 75, 0, 0.1)

batch_size = 64
n_epochs = 100
n_channels = train_set[0][0].shape[0]
# n_channels = X_train.shape[1]
n_classes = 4
model = CNN(n_channels,n_classes)
cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
  model.cuda()
clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(valid_set),  # using valid_set for validation
    # iterator_valid=val_dataloader,
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)
clf.fit(train_set,y=None,epochs=n_epochs)
Y = [x[1] for x in valid_set]
preds = clf.predict(valid_set)
M = (Y,preds)
np.save("S_all",M)
from google.colab import files
# files.download("S"+str(subject)+".npy")

  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.3931        1.5850            0.3329        1.4209  0.0006  9.3097
      2            0.4981        1.4067            0.3819        1.3272  0.0006  9.2936
      3            0.5505        1.2968            0.4336        1.2940  0.0006  9.4508
      4            0.5613        1.2249            0.4252        1.2739  0.0006  9.6690
      5            0.6223        1.1468            0.4799        1.1702  0.0006  9.9410
      6            0.6370        1.1074            0.4792        1.1985  0.0006  10.1657
      7            0.6065        1.0623            0.4522        1.2610  0.0006  10.1068
      8            0.6562        1.0255            0.4749        1.2197  0.0006  9.9660
      9            0.6786        0.9772            0.4942        1.1599  0.0006  9.8743
     10            0.6902     